# Hybrid network for classification

In [2]:
pip install lazypredict

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Imports
import tensorflow as tf
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from lazypredict.supervised import LazyClassifier
import numpy as np
import lazypredict

#from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score

/Users/manwest/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [54]:
df = pd.read_csv('/Users/manwest/Git/TINTOlib-benchmark/Datasets/covtype.csv')
#df['CLASSI_FIN'] = df.CLASSI_FIN.replace(to_replace=['arbovirus', 'inconclusive'], value=[1, 0])
df

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,3
581008,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,3
581009,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,3
581010,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,3


In [11]:
df

,RiskPerformance,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,...,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
0,1,55,144,4,84,20,3,0,83,2,...,43,0,0,0,33,-8,8,1,1,69
1,1,61,58,15,41,2,4,4,100,-7,...,67,0,0,0,0,-8,0,-8,-8,0
2,1,67,66,5,24,9,0,0,100,-7,...,44,0,4,4,53,66,4,2,1,86
3,1,66,169,1,73,28,1,1,93,76,...,57,0,5,4,72,83,6,4,3,91
4,1,81,333,27,132,12,0,0,100,-7,...,25,0,1,1,51,89,3,1,0,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10454,0,73,131,5,57,21,0,0,95,80,...,19,7,0,0,26,-8,5,2,0,100
10455,1,65,147,39,68,11,0,0,92,28,...,42,1,1,1,86,53,2,2,1,80
10456,1,74,129,6,64,18,1,1,100,-7,...,33,3,4,4,6,-8,5,-8,0,56
10457,1,72,234,12,113,42,2,2,96,35,...,20,6,0,0,19,-8,4,1,0,38


In [41]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = df.iloc[:,:-1]

X = scaler.fit_transform(X)
X

array([[ 0.10924527, -0.85658279, -0.91787643, ...,  0.07537121,
         0.04951605, -0.08449862],
       [ 0.10095249,  1.15315073, -0.24288289, ...,  0.07537121,
         0.04951605, -0.06146342],
       [ 0.27095451, -0.85658279, -0.24288289, ...,  0.07537121,
         0.04951605,  0.05831962],
       ...,
       [ 0.15900196, -0.85658279, -0.91787643, ...,  0.07537121,
         0.04951605,  0.56509403],
       [ 0.12168445, -0.85658279,  1.78209771, ...,  0.07537121,
         0.04951605, -0.08449862],
       [ 0.03046385,  1.15315073, -0.24288289, ...,  0.07537121,
         0.04951605, -0.07528454]])

In [45]:
y = df.iloc[:,-1]
y

0        1
1        1
2        1
3        1
4        1
        ..
17167    0
17168    0
17169    0
17170    0
17171    0
Name: CLASSI_FIN, Length: 17172, dtype: int64

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=64, shuffle=True)
X_train

array([[ 0.19217309, -0.85658279,  1.78209771, ...,  0.07537121,
         0.04951605, -0.06607046],
       [ 0.1424164 ,  1.15315073, -0.24288289, ...,  0.07537121,
         0.04951605, -0.0246071 ],
       [ 0.1424164 ,  1.15315073, -0.24288289, ...,  0.07537121,
         0.04951605, -0.07528454],
       ...,
       [ 0.03046385, -0.85658279, -0.24288289, ...,  0.07537121,
         0.04951605,  0.06292666],
       [ 0.16314835,  1.15315073, -0.24288289, ...,  0.07537121,
         0.04951605, -0.06146342],
       [ 0.22534421,  1.15315073, -0.24288289, ...,  0.07537121,
         0.04951605, -0.02921414]])

In [48]:
y_test

5800     1
1804     1
3108     1
9113     1
1575     1
        ..
7258     1
5759     1
15758    0
13267    0
13257    0
Name: CLASSI_FIN, Length: 5667, dtype: int64

In [49]:
#import lazypredict
lazypredict.supervised.CLASSIFIERS

[('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
 ('BaggingClassifier', sklearn.ensemble._bagging.BaggingClassifier),
 ('BernoulliNB', sklearn.naive_bayes.BernoulliNB),
 ('CalibratedClassifierCV', sklearn.calibration.CalibratedClassifierCV),
 ('CategoricalNB', sklearn.naive_bayes.CategoricalNB),
 ('DecisionTreeClassifier', sklearn.tree._classes.DecisionTreeClassifier),
 ('DummyClassifier', sklearn.dummy.DummyClassifier),
 ('ExtraTreeClassifier', sklearn.tree._classes.ExtraTreeClassifier),
 ('ExtraTreesClassifier', sklearn.ensemble._forest.ExtraTreesClassifier),
 ('GaussianNB', sklearn.naive_bayes.GaussianNB),
 ('KNeighborsClassifier',
  sklearn.neighbors._classification.KNeighborsClassifier),
 ('LabelPropagation',
  sklearn.semi_supervised._label_propagation.LabelPropagation),
 ('LabelSpreading', sklearn.semi_supervised._label_propagation.LabelSpreading),
 ('LinearDiscriminantAnalysis',
  sklearn.discriminant_analysis.LinearDiscriminantAnalysis),
 ('Linear

In [9]:
#lazypredict.supervised.REGRESSORS = lazypredict.supervised.REGRESSORS[33:36]
#lazypredict.supervised.REGRESSORS

[('RandomForestRegressor', sklearn.ensemble._forest.RandomForestRegressor),
 ('Ridge', sklearn.linear_model._ridge.Ridge),
 ('RidgeCV', sklearn.linear_model._ridge.RidgeCV)]

In [9]:
lazypredict.supervised.REGRESSORS = lazypredict.supervised.REGRESSORS[36:42]
lazypredict.supervised.REGRESSORS

[('SGDRegressor', sklearn.linear_model._stochastic_gradient.SGDRegressor),
 ('SVR', sklearn.svm._classes.SVR),
 ('TransformedTargetRegressor',
  sklearn.compose._target.TransformedTargetRegressor),
 ('TweedieRegressor', sklearn.linear_model._glm.glm.TweedieRegressor),
 ('XGBRegressor', xgboost.sklearn.XGBRegressor),
 ('LGBMRegressor', lightgbm.sklearn.LGBMRegressor)]

In [50]:
#regressor=['RandomForestRegressor', 'XGBRegressor', 'GradientBoostingRegressor', 'HistGradientBoostingRegressor']
reg = LazyClassifier(verbose=2, ignore_warnings=True, custom_metric=None)

In [51]:
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

  7%|▋         | 2/29 [00:00<00:05,  5.27it/s]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.7390153520381154, 'Balanced Accuracy': 0.6782198574877074, 'ROC AUC': 0.6782198574877073, 'F1 Score': 0.7303300003733973, 'Time taken': 0.20759797096252441}
{'Model': 'BaggingClassifier', 'Accuracy': 0.718369507676019, 'Balanced Accuracy': 0.6823946872753496, 'ROC AUC': 0.6823946872753495, 'F1 Score': 0.7187496727576611, 'Time taken': 0.17708492279052734}
{'Model': 'BernoulliNB', 'Accuracy': 0.7353096876654315, 'Balanced Accuracy': 0.6944692410216013, 'ROC AUC': 0.6944692410216015, 'F1 Score': 0.7337045639657988, 'Time taken': 0.02405691146850586}


 28%|██▊       | 8/29 [00:00<00:01, 17.45it/s]

{'Model': 'CalibratedClassifierCV', 'Accuracy': 0.7353096876654315, 'Balanced Accuracy': 0.6719029843166509, 'ROC AUC': 0.6719029843166509, 'F1 Score': 0.7256083093129488, 'Time taken': 0.10647106170654297}
{'Model': 'DecisionTreeClassifier', 'Accuracy': 0.6739015352038116, 'Balanced Accuracy': 0.637364377456695, 'ROC AUC': 0.6373643774566949, 'F1 Score': 0.6759496740220255, 'Time taken': 0.05228590965270996}
{'Model': 'DummyClassifier', 'Accuracy': 0.6710781718722428, 'Balanced Accuracy': 0.5, 'ROC AUC': 0.5, 'F1 Score': 0.5389884451172416, 'Time taken': 0.009890079498291016}
{'Model': 'ExtraTreeClassifier', 'Accuracy': 0.6779601199929416, 'Balanced Accuracy': 0.643123609777237, 'ROC AUC': 0.643123609777237, 'F1 Score': 0.6803066285859849, 'Time taken': 0.03048396110534668}


 38%|███▊      | 11/29 [00:01<00:02,  7.11it/s]

...

{'Model': 'SGDClassifier', 'Accuracy': 0.7051349920592906, 'Balanced Accuracy': 0.6710296338219545, 'ROC AUC': 0.6710296338219545, 'F1 Score': 0.7065377795994867, 'Time taken': 0.11475205421447754}


 90%|████████▉ | 26/29 [00:14<00:02,  1.10it/s]

{'Model': 'SVC', 'Accuracy': 0.7388388918298924, 'Balanced Accuracy': 0.6731648354190671, 'ROC AUC': 0.6731648354190671, 'F1 Score': 0.7280867907788116, 'Time taken': 3.161607265472412}
{'Model': 'XGBClassifier', 'Accuracy': 0.7414857949532381, 'Balanced Accuracy': 0.6892237774785888, 'ROC AUC': 0.6892237774785887, 'F1 Score': 0.7359950067630042, 'Time taken': 0.08319282531738281}
[LightGBM] [Info] Number of positive: 7645, number of negative: 3860
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001706 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 440
[LightGBM] [Info] Number of data points in the train set: 11505, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.664494 -> initscore=0.683385
[LightGBM] [Info] Start training from score 0.683385


100%|██████████| 29/29 [00:14<00:00,  1.94it/s]

{'Model': 'LGBMClassifier', 'Accuracy': 0.7476619022410447, 'Balanced Accuracy': 0.6969710071899415, 'ROC AUC': 0.6969710071899416, 'F1 Score': 0.7425611532666628, 'Time taken': 0.5064549446105957}


In [53]:
models


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NearestCentroid,0.70,0.70,0.70,0.71,0.04
LGBMClassifier,0.75,0.70,0.70,0.74,0.51
BernoulliNB,0.74,0.69,0.69,0.73,0.02
RandomForestClassifier,0.74,0.69,0.69,0.74,0.73
XGBClassifier,0.74,0.69,0.69,0.74,0.08
GaussianNB,0.71,0.68,0.68,0.71,0.01
BaggingClassifier,0.72,0.68,0.68,0.72,0.18
AdaBoostClassifier,0.74,0.68,0.68,0.73,0.21
KNeighborsClassifier,0.73,0.68,0.68,0.72,0.06
